<a href="https://www.kaggle.com/code/marianameyerf/notebook40af724143?scriptVersionId=110443342" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BR_BERTo_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

BR_BERTo

CENTENFolha - 8 partes

Batch size = 16

GPU T4x2

(07h:21m)

In [2]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.6.4
Uninstalling tensorflow-2.6.4:
  Successfully uninstalled tensorflow-2.6.4
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4irw07di
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-4irw07di
  Resolved https://github.com/huggingface/transformers to commit 258963062bc09bb412dbbdbb777632b4057a1556
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5554323 sha256=f39225b73f0030b1a38a1ca2769a852ddc7572feac93738b22f3bf0c56cb54b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-3hfcy0yu/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Unin

In [3]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
#!curl -L  https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
!curl -L https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

!awk NF < dump.txt > kant.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3724k  100 3724k    0     0  7419k      0 --:--:-- --:--:-- --:--:-- 7404k


In [4]:
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ac
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ad
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ae 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_af
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ag
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ah

--2022-11-08 14:45:16--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [text/plain]
Saving to: ‘cetenfolha_aa.12’

cetenfolha_aa.12    100%[===================>]   3.64M  --.-KB/s    in 0.07s   

2022-11-08 14:45:16 (53.2 MB/s) - ‘cetenfolha_aa.12’ saved [3813899/3813899]

--2022-11-08 14:45:17--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [

In [5]:
!cat cetenfolha_aa cetenfolha_ab cetenfolha_ac cetenfolha_ad cetenfolha_ae cetenfolha_af cetenfolha_ag cetenfolha_ah>  dump.txt
!awk NF < dump.txt > kant.txt

In [6]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695M [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-e454ccb8a36a9604/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
#@title Step 10: Building the Dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./kant.txt",
    block_size=32,
)

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [10]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

In [11]:
tokenized_datasets["train"][4]

{'input_ids': [0,
  225,
  51,
  1479,
  321,
  225,
  39,
  83,
  1264,
  357,
  1050,
  57714,
  560,
  13220,
  1199,
  355,
  1473,
  268,
  3976,
  225,
  12,
  5914,
  21628,
  16,
  10865,
  425,
  9250,
  13,
  268,
  806,
  301,
  5600,
  268,
  2601,
  2166,
  920,
  2771,
  328,
  1356,
  439,
  695,
  734,
  16,
  318,
  683,
  268,
  5237,
  495,
  683,
  16,
  301,
  346,
  276,
  301,
  1560,
  328,
  1356,
  4476,
  225,
  18,
  225,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [12]:
#block_size = tokenizer.model_max_length
block_size = 32

In [13]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

In [15]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [16]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [17]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BR_BERTo-CENTENFolha-8_parts",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

In [18]:
#@title Step 13: Pre-training the Model
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 219622
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 68640
  Number of trainable parameters = 173042160
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,4.335200
1000,3.845400
1500,3.699900
2000,3.483400
2500,3.333500
3000,3.260100
3500,3.191800
4000,3.127000
4500,3.108600
5000,3.060500


Saving model checkpoint to ./BR_BERTo-CENTENFolha-8_parts/checkpoint-10000
Configuration saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-10000/config.json
Model weights saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-10000/pytorch_model.bin
Deleting older checkpoint [BR_BERTo-CENTENFolha-8_parts/checkpoint-60000] due to args.save_total_limit
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./BR_BERTo-CENTENFolha-8_parts/checkpoint-20000
Configuration saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-20000/config.json
Model weights saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-20000/pytorch_model.bin
Deleting older checkpoint [BR_BERTo-CENTENFolha-8_parts/checkpoint-50000] due to args.save_total_limit
/opt/conda/lib/python3.7/si

TrainOutput(global_step=68640, training_loss=2.420741958884926, metrics={'train_runtime': 25420.9365, 'train_samples_per_second': 86.394, 'train_steps_per_second': 2.7, 'total_flos': 2.42237682625536e+16, 'train_loss': 2.420741958884926, 'epoch': 10.0})

In [19]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
#trainer.save_model("./KantaiBERT")
trainer.save_model('./BR_BERTo-CENTENFolha-8_parts')

Saving model checkpoint to ./BR_BERTo-CENTENFolha-8_parts
Configuration saved in ./BR_BERTo-CENTENFolha-8_parts/config.json
Model weights saved in ./BR_BERTo-CENTENFolha-8_parts/pytorch_model.bin


In [20]:
# Saving model on Wandb
import wandb
wandb.save('BR_BERTo-CENTENFolha-8_parts.h5')

[]

In [21]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BR_BERTo-CENTENFolha-8_parts",
    tokenizer=tokenizer
)

loading configuration file ./BR_BERTo-CENTENFolha-8_parts/config.json
Model config RobertaConfig {
  "_name_or_path": "./BR_BERTo-CENTENFolha-8_parts",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 8,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 150000
}

loading configuration file ./BR_BERTo-CENTENFolha-8_parts/config.json
Model config RobertaConfig {
  "_name_or_path": "./BR_BERTo-CENTENFolha-8_parts",
  "architectures": [

In [22]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 72871
  Batch size = 16


Perplexity: 8.34


In [23]:
eval_results

{'eval_loss': 2.1214816570281982,
 'eval_runtime': 474.5314,
 'eval_samples_per_second': 153.564,
 'eval_steps_per_second': 9.599,
 'epoch': 10.0}

In [24]:
fill_mask("O rapaz olhou para o <mask> ")

[{'score': 0.3879077434539795,
  'token': 225,
  'token_str': ' ',
  'sequence': 'O rapaz olhou para o  '},
 {'score': 0.06767208874225616,
  'token': 6358,
  'token_str': ' banheiro',
  'sequence': 'O rapaz olhou para o banheiro '},
 {'score': 0.03540477156639099,
  'token': 5,
  'token_str': '!',
  'sequence': 'O rapaz olhou para o! '},
 {'score': 0.028271976858377457,
  'token': 35,
  'token_str': '?',
  'sequence': 'O rapaz olhou para o? '},
 {'score': 0.02615528367459774,
  'token': 3231,
  'token_str': ' quarto',
  'sequence': 'O rapaz olhou para o quarto '}]

In [25]:
fill_mask("A moça olhou para o <mask> ")

[{'score': 0.377534419298172,
  'token': 225,
  'token_str': ' ',
  'sequence': 'A moça olhou para o  '},
 {'score': 0.06184181571006775,
  'token': 6358,
  'token_str': ' banheiro',
  'sequence': 'A moça olhou para o banheiro '},
 {'score': 0.03320068493485451,
  'token': 35,
  'token_str': '?',
  'sequence': 'A moça olhou para o? '},
 {'score': 0.02839978225529194,
  'token': 5,
  'token_str': '!',
  'sequence': 'A moça olhou para o! '},
 {'score': 0.02712242864072323,
  'token': 30,
  'token_str': ':',
  'sequence': 'A moça olhou para o: '}]

In [26]:
fill_mask("Comprou uma <mask> na loja. ")

[{'score': 0.023507224395871162,
  'token': 5305,
  'token_str': ' pedra',
  'sequence': 'Comprou uma pedra na loja. '},
 {'score': 0.0184037946164608,
  'token': 7401,
  'token_str': ' árvore',
  'sequence': 'Comprou uma árvore na loja. '},
 {'score': 0.017729027196764946,
  'token': 3363,
  'token_str': ' nota',
  'sequence': 'Comprou uma nota na loja. '},
 {'score': 0.016659963876008987,
  'token': 3404,
  'token_str': ' loja',
  'sequence': 'Comprou uma loja na loja. '},
 {'score': 0.01600511558353901,
  'token': 2620,
  'token_str': ' mão',
  'sequence': 'Comprou uma mão na loja. '}]

In [27]:
fill_mask("A mulher não é <mask>. ")

[{'score': 0.9998087286949158,
  'token': 225,
  'token_str': ' ',
  'sequence': 'A mulher não é. '},
 {'score': 3.2044121326180175e-05,
  'token': 18,
  'token_str': '.',
  'sequence': 'A mulher não é.. '},
 {'score': 9.194252015731763e-06,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'A mulher não é «. '},
 {'score': 5.893655270483578e-06,
  'token': 3817,
  'token_str': ' tel',
  'sequence': 'A mulher não é tel. '},
 {'score': 5.698397671949351e-06,
  'token': 5,
  'token_str': '!',
  'sequence': 'A mulher não é!. '}]

In [28]:
fill_mask("O homem não é <mask>. ")

[{'score': 0.9995953440666199,
  'token': 225,
  'token_str': ' ',
  'sequence': 'O homem não é. '},
 {'score': 8.137679833453149e-05,
  'token': 18,
  'token_str': '.',
  'sequence': 'O homem não é.. '},
 {'score': 1.6248592146439478e-05,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'O homem não é «. '},
 {'score': 1.2915032129967585e-05,
  'token': 5,
  'token_str': '!',
  'sequence': 'O homem não é!. '},
 {'score': 1.2408900147420354e-05,
  'token': 261,
  'token_str': ' d',
  'sequence': 'O homem não é d. '}]

In [29]:
fill_mask("Ele é um bom <mask>. ")

[{'score': 0.9997895359992981,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Ele é um bom. '},
 {'score': 0.00014541717246174812,
  'token': 18,
  'token_str': '.',
  'sequence': 'Ele é um bom.. '},
 {'score': 6.266754553507781e-06,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Ele é um bom «. '},
 {'score': 2.9389618703135056e-06,
  'token': 272,
  'token_str': ' m',
  'sequence': 'Ele é um bom m. '},
 {'score': 2.72923330157937e-06,
  'token': 586,
  'token_str': ' dia',
  'sequence': 'Ele é um bom dia. '}]

In [30]:
fill_mask("Ela é uma boa <mask>. ")

[{'score': 0.9998600482940674,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Ela é uma boa. '},
 {'score': 6.6301719925832e-05,
  'token': 18,
  'token_str': '.',
  'sequence': 'Ela é uma boa.. '},
 {'score': 7.274513791344361e-06,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Ela é uma boa «. '},
 {'score': 2.6879301913140807e-06,
  'token': 5,
  'token_str': '!',
  'sequence': 'Ela é uma boa!. '},
 {'score': 2.2841427380626556e-06,
  'token': 2654,
  'token_str': ' »',
  'sequence': 'Ela é uma boa ». '}]

In [31]:
fill_mask("Faz de conta que ainda é <mask>. ")

[{'score': 0.9996553659439087,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Faz de conta que ainda é. '},
 {'score': 0.00011703727795975283,
  'token': 18,
  'token_str': '.',
  'sequence': 'Faz de conta que ainda é.. '},
 {'score': 2.3654527467442676e-05,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Faz de conta que ainda é «. '},
 {'score': 1.17270055852714e-05,
  'token': 1208,
  'token_str': ' bom',
  'sequence': 'Faz de conta que ainda é bom. '},
 {'score': 5.484273060574196e-06,
  'token': 5,
  'token_str': '!',
  'sequence': 'Faz de conta que ainda é!. '}]

In [32]:
fill_mask("Depois da tempestade vem a <mask>. ")

[{'score': 0.930250883102417,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Depois da tempestade vem a. '},
 {'score': 0.003178785787895322,
  'token': 3257,
  'token_str': ' pé',
  'sequence': 'Depois da tempestade vem a pé. '},
 {'score': 0.0022572630550712347,
  'token': 18,
  'token_str': '.',
  'sequence': 'Depois da tempestade vem a.. '},
 {'score': 0.0016144736437126994,
  'token': 752,
  'token_str': ' todos',
  'sequence': 'Depois da tempestade vem a todos. '},
 {'score': 0.0014756583841517568,
  'token': 1489,
  'token_str': ' fim',
  'sequence': 'Depois da tempestade vem a fim. '}]

In [33]:
fill_mask("Mais vale um pássaro na mão do que <mask> voando. ")

[{'score': 0.047030456364154816,
  'token': 575,
  'token_str': ' eu',
  'sequence': 'Mais vale um pássaro na mão do que eu voando. '},
 {'score': 0.04598027095198631,
  'token': 506,
  'token_str': ' ele',
  'sequence': 'Mais vale um pássaro na mão do que ele voando. '},
 {'score': 0.027364060282707214,
  'token': 769,
  'token_str': ' ela',
  'sequence': 'Mais vale um pássaro na mão do que ela voando. '},
 {'score': 0.011642674915492535,
  'token': 1139,
  'token_str': ' outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando. '},
 {'score': 0.010727044194936752,
  'token': 582,
  'token_str': ' você',
  'sequence': 'Mais vale um pássaro na mão do que você voando. '}]

In [34]:
fill_mask("Não tinha <mask>, mas almoçou mesmo assim. ")

[{'score': 0.04077940806746483,
  'token': 1976,
  'token_str': ' dinheiro',
  'sequence': 'Não tinha dinheiro, mas almoçou mesmo assim. '},
 {'score': 0.034136999398469925,
  'token': 734,
  'token_str': ' tempo',
  'sequence': 'Não tinha tempo, mas almoçou mesmo assim. '},
 {'score': 0.017651768401265144,
  'token': 2541,
  'token_str': ' filhos',
  'sequence': 'Não tinha filhos, mas almoçou mesmo assim. '},
 {'score': 0.012891470454633236,
  'token': 1428,
  'token_str': ' nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim. '},
 {'score': 0.012331949546933174,
  'token': 4374,
  'token_str': ' jeito',
  'sequence': 'Não tinha jeito, mas almoçou mesmo assim. '}]

In [35]:
fill_mask("Bebeu um copo d'água, pois tinha <mask>. ")

[{'score': 0.9996600151062012,
  'token': 225,
  'token_str': ' ',
  'sequence': "Bebeu um copo d'água, pois tinha. "},
 {'score': 0.00018398030078969896,
  'token': 18,
  'token_str': '.',
  'sequence': "Bebeu um copo d'água, pois tinha.. "},
 {'score': 5.394807885750197e-05,
  'token': 735,
  'token_str': ' min',
  'sequence': "Bebeu um copo d'água, pois tinha min. "},
 {'score': 5.774219062004704e-06,
  'token': 3817,
  'token_str': ' tel',
  'sequence': "Bebeu um copo d'água, pois tinha tel. "},
 {'score': 4.859412911173422e-06,
  'token': 261,
  'token_str': ' d',
  'sequence': "Bebeu um copo d'água, pois tinha d. "}]

In [36]:
fill_mask("Sem saber, ele descobriu um <mask>. ")

[{'score': 0.9903740882873535,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Sem saber, ele descobriu um. '},
 {'score': 0.0003562170313671231,
  'token': 18,
  'token_str': '.',
  'sequence': 'Sem saber, ele descobriu um.. '},
 {'score': 0.00024357213987968862,
  'token': 586,
  'token_str': ' dia',
  'sequence': 'Sem saber, ele descobriu um dia. '},
 {'score': 0.00016274995869025588,
  'token': 4310,
  'token_str': ' telefone',
  'sequence': 'Sem saber, ele descobriu um telefone. '},
 {'score': 0.00016093242447823286,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Sem saber, ele descobriu um «. '}]

In [37]:
fill_mask("Pedro fez fortuna vendendo <mask>. ")

[{'score': 0.9995922446250916,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Pedro fez fortuna vendendo. '},
 {'score': 0.00011580066347960383,
  'token': 18,
  'token_str': '.',
  'sequence': 'Pedro fez fortuna vendendo.. '},
 {'score': 5.87530157645233e-05,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Pedro fez fortuna vendendo «. '},
 {'score': 1.9351051378180273e-05,
  'token': 2654,
  'token_str': ' »',
  'sequence': 'Pedro fez fortuna vendendo ». '},
 {'score': 1.542612153571099e-05,
  'token': 735,
  'token_str': ' min',
  'sequence': 'Pedro fez fortuna vendendo min. '}]

In [38]:
fill_mask("Sem medo de ser <mask>. ")

[{'score': 0.9991890788078308,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Sem medo de ser. '},
 {'score': 8.056305523496121e-05,
  'token': 18,
  'token_str': '.',
  'sequence': 'Sem medo de ser.. '},
 {'score': 5.334976958693005e-05,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Sem medo de ser «. '},
 {'score': 1.8366428776062094e-05,
  'token': 2837,
  'token_str': ' etc',
  'sequence': 'Sem medo de ser etc. '},
 {'score': 1.6314515960402787e-05,
  'token': 5,
  'token_str': '!',
  'sequence': 'Sem medo de ser!. '}]

In [39]:
wandb.finish()

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
